### read in kadonis data

In [1]:
import os, glob
import numpy as np
import pandas as pd

import plotly
import plotly.plotly as py

import json
import requests
from requests.auth import HTTPBasicAuth

In [2]:
ag_kad_file = './kad_files/113In-ag-YGO09'
an_kad_file = './kad_files/113In-an-YGO09'

In [3]:
def get_kadonis_df(filename):
    # open kadonis file
    f = open(filename,'r')
    # read lines and close
    lines = f.read().splitlines()
    f.close()
    # strip out lines we want
    lines_imp = lines[19:]
    x = [line.split() for line in lines_imp]
    # get column labels, column data
    headers = x[0]
    data = x[2:]
    # convert to df
    df = pd.DataFrame(data, columns=headers)
    # make numeric
    ene  = pd.to_numeric(df['E(c.m.)'])
    cs_true = pd.to_numeric(df['CS'])
    cs_err  = pd.to_numeric(df['ErrCS'])
    # new df  
    df_new  = pd.DataFrame([ene, cs_true, cs_err]).transpose()
    df_new.columns = ['ene', 'exp_cs', 'cs_err']
    df_new = df_new.sort_values('ene', ascending=True)
    return(df_new)

In [4]:
ag_kad_df = get_kadonis_df(ag_kad_file)
an_kad_df = get_kadonis_df(an_kad_file)
print(ag_kad_df)
print(an_kad_df)

       ene    exp_cs        cs_err
0    8.660  0.000004  5.000000e-07
2    9.147  0.000007  6.000000e-07
1    9.153  0.000006  6.000000e-07
3    9.553  0.000015  1.000000e-06
4    9.660  0.000020  2.000000e-06
5   10.187  0.000037  3.000000e-06
6   10.606  0.000064  5.000000e-06
7   10.704  0.000070  6.000000e-06
8   11.085  0.000111  9.000000e-06
9   11.573  0.000200  1.600000e-05
10  12.162  0.000341  2.800000e-05
11  12.536  0.000435  3.500000e-05
12  13.018  0.000588  4.700000e-05
13  13.640  0.000745  6.000000e-05
      ene   exp_cs   cs_err
0   9.660  0.00007  0.00002
1  10.187  0.00027  0.00004
2  10.606  0.00082  0.00007
3  10.704  0.00100  0.00008
4  11.085  0.00210  0.00020
5  11.573  0.00600  0.00050
6  12.162  0.01300  0.00100
7  12.536  0.02500  0.00200
8  13.018  0.05000  0.00400
9  13.640  0.08800  0.00600


### start talys stuff

In [5]:
symbol = 'in'
mass = 113
z = 49

In [6]:
# write ag_energies file
ene_kad = []
[ene_kad.append(round(ene,4)) for ene in ag_kad_df['ene']]
[ene_kad.append(round(ene,4)) for ene in an_kad_df['ene']]
ene_kad = sorted(list(set(ene_kad)))
ene_kad

[8.66,
 9.147,
 9.153,
 9.553,
 9.66,
 10.187,
 10.606,
 10.704,
 11.085,
 11.573,
 12.162,
 12.536,
 13.018,
 13.64]

In [7]:
f = open('energies','w')
[f.write(str(ene)+'\n') for ene in ene_kad]
f.close()

In [8]:
def get_talys_df(filename):
    f = open(filename,'r')
    # read lines and close
    lines = f.read().splitlines()
    f.close()
    # strip out lines we want
    lines_imp = lines[5:]
    x = [line.strip().split(' ') for line in lines_imp]
    df = pd.DataFrame(x)
    df.columns = ['ene','cs']
    # make numeric 
    df  = pd.DataFrame([pd.to_numeric(df['ene']),
                        # convert millibarns
                        pd.to_numeric(df['cs'])/1000]).transpose()
    return(df)

In [9]:
ag_talys_file = 'rp0'+str(z+2)+str(mass+4)+'.tot'
an_talys_file = 'rp0'+str(z+2)+str(mass+3)+'.tot'

In [10]:
# ldmodel = [1,2,3,4,5,6]
# strength = [1,2,3,4,5,6,7,8]
# alphaomp = [1,2,3,4,5,6,7,8]
ldmodel = [1,2]
strength = [1,2]
alphaomp = [1,2]
jlmomp = ['y','n']

ag_talys_all_df = pd.DataFrame(ene_kad)
an_talys_all_df = pd.DataFrame(ene_kad)

In [11]:
col_names = ['ene']

num_iter = len(ldmodel)*len(strength)*len(alphaomp)*len(jlmomp)
iters_done = 0

for l in ldmodel:
    for s in strength:
        for a in alphaomp:
            for j in jlmomp:

                # make column name
                col_name = 'l'+str(l)+'s'+str(s)+'a'+str(a)+'j'+j
                col_names.append(col_name)
                
                print('Starting {}'.format(col_name))
                
                # write input file contents
                top = 'projectile a\nelement '+symbol+'\nmass '+str(mass)+'\nenergy energies\n'
                mid = 'ldmodel '+str(l)+'\nstrength '+str(s)+'\nalphaomp '+str(a)+'\njlmomp '+j+'\n'
                bottom = 'xseps 1.e-35\npopeps 1.e-35\ntranseps 1.e-35\ncbreak p 0.\ncbreak n 0.\ncstrip p 0.\ncstrip n 0.\ncknock p 0.\ncknock n 0.\ngnorm 1'
                
                # actually write input file
                f = open('input', 'w')
                f.write(top+mid+bottom)
                f.close()

                print('Input file created.\nRunning talys...')
                
                # run talys
                !talys <input> output

                print('Done with {}\n'.format(col_name))
                 
                # concatenate
                ag_talys_all_df = pd.concat([ag_talys_all_df, get_talys_df(ag_talys_file)['cs']], axis=1)
                an_talys_all_df = pd.concat([an_talys_all_df, get_talys_df(an_talys_file)['cs']], axis=1)
                
                iters_done = iters_done + 1
                print('PROGRESS UPDATE: {}%\n'.format((iters_done/num_iter)*100))
            
                
print('Done with all talys calculations.\nCleaning file system...')

keeping = [ag_talys_file, an_talys_file, 'output', 'total.tot', '113-indium.ipynb']
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    if(f not in keeping):
        os.remove(f)

print('Done cleaning files.')

Starting l1s1a1jy
Input file created.
Running talys...
Done with l1s1a1jy

PROGRESS UPDATE: 6.25%

Starting l1s1a1jn
Input file created.
Running talys...
Done with l1s1a1jn

PROGRESS UPDATE: 12.5%

Starting l1s1a2jy
Input file created.
Running talys...
Done with l1s1a2jy

PROGRESS UPDATE: 18.75%

Starting l1s1a2jn
Input file created.
Running talys...
Done with l1s1a2jn

PROGRESS UPDATE: 25.0%

Starting l1s2a1jy
Input file created.
Running talys...
Done with l1s2a1jy

PROGRESS UPDATE: 31.25%

Starting l1s2a1jn
Input file created.
Running talys...
Done with l1s2a1jn

PROGRESS UPDATE: 37.5%

Starting l1s2a2jy
Input file created.
Running talys...
Done with l1s2a2jy

PROGRESS UPDATE: 43.75%

Starting l1s2a2jn
Input file created.
Running talys...
Done with l1s2a2jn

PROGRESS UPDATE: 50.0%

Starting l2s1a1jy
Input file created.
Running talys...
Done with l2s1a1jy

PROGRESS UPDATE: 56.25%

Starting l2s1a1jn
Input file created.
Running talys...
Done with l2s1a1jn

PROGRESS UPDATE: 62.5%

Starti

In [12]:
ag_talys_all_df.columns = col_names
an_talys_all_df.columns = col_names

In [13]:
# merge talys with kadonis
ag_mega_df = pd.merge_asof(ag_talys_all_df, ag_kad_df).dropna().set_index('ene')
an_mega_df = pd.merge_asof(an_talys_all_df, an_kad_df).dropna().set_index('ene')

In [18]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [45]:
rmse_vals = []
for col in ag_mega_df.drop('cs_err',axis=1):
    predictions = ag_mega_df[col].values
    targets = ag_mega_df['cs_err']
    rmse_vals.append(rmse(predictions,targets))
lowest_rmse = ag_mega_df.columns[rmse_vals.index(min(rmse_vals))]
ld = lowest_rmse[1]
st = lowest_rmse[3]
al = lowest_rmse[5]
jl = lowest_rmse[7]
print('Following parameters yield lowest RMSE:\n* * * * * * * * * * * * * * * * * * * *\nldmodel: {}\nstrength: {}\nalphaomp: {}\njlmomp: {}'.format(ld, st, al, jl))

Following parameters yield lowest RMSE:
* * * * * * * * * * * * * * * * * * * *
ldmodel: 2
strength: 1
alphaomp: 2
jlmomp: y


### plotting

In [15]:
def clear_plotly():
    username = 'ptinsley' # Replace with YOUR USERNAME
    api_key = 'zAX8msTnvXRmY2PcMcSE' # Replace with YOUR API KEY
    auth = HTTPBasicAuth(username, api_key)
    headers = {'Plotly-Client-Platform': 'python'}
    plotly.tools.set_credentials_file(username=username, api_key=api_key)
    def get_pages(username, page_size):
        url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
        response = requests.get(url, auth=auth, headers=headers)
        if response.status_code != 200:
            return
        page = json.loads(response.content)
        yield page
        while True:
            resource = page['children']['next'] 
            if not resource:
                break
            response = requests.get(resource, auth=auth, headers=headers)
            if response.status_code != 200:
                break
            page = json.loads(response.content)
            yield page
    def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
        for page in get_pages(username, page_size):
            for x in range(0, len(page['children']['results'])):
                fid = page['children']['results'][x]['fid']
                res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
                res.raise_for_status()
                if res.status_code == 200:
                    json_res = json.loads(res.content)
                    if json_res['filetype'] == filetype_to_delete:
                        # move to trash
                        requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                        # permanently delete
                        requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)
    permanently_delete_files('ptinsley', filetype_to_delete='plot')
    permanently_delete_files('ptinsley', filetype_to_delete='grid')